# Explore Natural vs. Anthropogenic CO$_{2}$
- After ICDC, we realized that it isn't exactly clear what our residuals represent. For awhile, we portrayed them as simply "natural CO$_{2}$." We realized recently, that embedded in the ensemble mean (forced signal) is a strong seasonal cycle. The trend within the ensemble mean is likely the anthropogenic invasion of carbon, but what is this seasonal cycle? Surely it isn't purely anthropogenic carbon, so these residuals more likely represent variability in natural CO2 due to internally generated variability. 
- The goal of this notebook is to use FG_ALT_CO2 output to understand what all of our time series components represent. The same processing has been done to the FG_ALT_CO2 output as other variables: it has been area-weighted over each region and split into the ensemble mean and residuals. The output itself was generated with the same dynamics of each run but with constant pre-industrial CO2 levels. Thus, it represents purely natural CO2 flux. We can take FG_CO2 - FG_ALT_CO2 to accurately extract anthropogenic carbon.
- The questions we are trying to answer here are:

1. What do the residuals represent from the FG_CO2 runs?
2. Is there any trend in the FG_ALT_CO2 timeseries?
3. Does the seasonal cycle show up in the FG_ALT_CO2 output? What drives that seasonal cycle?

Once we answer these fundamental questions, we can go on to really quantify and characterize the full time series from the historical runs.

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
%matplotlib inline

# Set Parameters

In [7]:
EBU = "CalCS"

# Functions

In [8]:
def load_in_filtered_data(VAR, EBU, AW=True):
    fileDir = '/glade/p/work/rbrady/EBUS_BGC_Variability/' + VAR + '/' + EBU + '/filtered_output/'
    if AW == True:
        ds_forced = xr.open_dataset(fileDir + EBU.lower() + '-' + VAR + '-forced-signal-AW-chavez-800km.nc')
        ds_residuals = xr.open_dataset(fileDir + EBU.lower() + '-' + VAR + '-residuals-AW-chavez-800km.nc')
    if AW == False:
        ds_forced = xr.open_dataset(fileDir + EBU.lower() + '-' + VAR + '-forced-signal-chavez-800km.nc')
        ds_residuals = xr.open_dataset(fileDir + EBU.lower() + '-' + VAR + '-residuals-chavez-800km.nc')
    ds_total = ds_forced + ds_residuals
    return ds_forced, ds_residuals, ds_total

# Load in FG_ALT_CO2 and FG_CO2 Output

In [12]:
alt_forced, alt_residuals, alt_total = load_in_filtered_data('FG_ALT_CO2', EBU)
contemp_forced, contemp_residuals, contemp_total = load_in_filtered_data('FG_CO2', EBU)